Neill Shikada and Melanie Sharif

# Import Dependencies
1. 

In [1]:
!pip install earthengine-api --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 244 kB 6.5 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.334-py3-none-any.whl size=274127 sha256=d24271646671b53aab7c7e14522175dbd805afdca811778c075ae9d10a752622
  Stored in directory: /root/.cache/pip/wheels/0a/d9/6b/1601dd4443ffced1345b91e6635c334a0c21f492a180c1ad6a
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.332
    Uninstalling earthengine-api-0.1.332:
      Successfully uninstalled earthengine-api-0.1.332
